In [28]:
import mediapipe as mp 
import numpy as np 
import cv2

In [29]:
mp_drawing = mp.solutions.drawing_utils
mp_pose    = mp.solutions.pose
mp_drawing_styles = mp.solutions.drawing_styles

normal_low, normal_hi = 178, 185
min_valgum            = 10
max_valugm            = 20 
min_varum             = 10 
max_varum             = 20 

frame_path = 'images/test1.jpg'
frame      = cv2.imread(frame_path)

In [30]:
# Functions


def round_up_2dec(x):
    return np.ceil(x * 100) / 100


def calculate_angle(a , b , c) : 

    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    ab = a - b
    cb = c - b 

    radians = np.arctan2(c[1] - b[1] , c[0]-b[0]) - np.arctan2(a[1] - b[1] , a[0]-b[0])
    angle   = np.abs(radians * 180.0 / np.pi)

    return angle
        

def classify_tibiofemoral_angle(angle):


    
    if angle < normal_low:
        # Genu Valgum
        deviation = normal_low - angle
        if deviation < min_valgum:            # If dervation is less than 10 degree -> detected as mild 
            return 'Valgum' , 'Mild', 1
        elif deviation <= max_valugm:         # If derivatoin is Between 10 , 20 degree -> detected as moderate
            return 'Valgum' , 'Moderate' , 2
        else:
             return 'Valgum' , 'Severe' , 3          #If dervation is less than 10 degree -> detected as mild 
    
    elif angle > normal_hi:
        # Genu Varum
        deviation = angle - normal_hi
        if deviation < min_varum:            # If dervation is less than 10 degree -> detected as mild 
            return 'Varum' , 'Mild' , 1
        elif deviation <= max_varum:         #If derivatoin is Between 10 , 20 degree -> detected as moderate
            return 'Varum' , 'Modearte', 2
        else:
                       
            return 'Varum', 'Severe',3       #If dervation is less than 10 degree -> detected as mild 
    else:
       
        return "Normal" ,'Normal', 0 



In [32]:
with mp_pose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:

        frame = cv2.imread('path_to_image.jpg')
        if frame is None:
            print("⚠️ Image not found or path incorrect")
            exit() 
                
        mp_drawing.draw_landmarks(frame , results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
        
        
        if results.pose_landmarks:
            
            landmarks = results.pose_landmarks.landmark
                
            # Capture left landmarks
            hip_l = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
        
            knee_l = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                      landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            
            ankle_l = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                       landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
        
            # Capture Right landmarks
            hip_r = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                     landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            
            knee_r = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                      landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            
            ankle_r = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                       landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
        
            # Calculate angles
            angle_left  = round_up_2dec(calculate_angle(hip_l, knee_l, ankle_l))
            angle_right = round_up_2dec(calculate_angle(hip_r, knee_r, ankle_r))

            
        
            # Classifying
            
            type_l , sev_l , state_l = classify_tibiofemoral_angle(angle_left)
            type_r , sev_r , state_r = classify_tibiofemoral_angle(angle_right)

            if state_l > state_r:

                final_type = type_l
                final_sev = sev_l
                
            elif state_r > state_l :
                final_type = type_r
                final_sev  = sev_r
                
                
            else : 
                
                final_type = type_l
                final_sev  = ''


            cv2.putText(frame , str(angle_left) , 
                       tuple(np.multiply(knee_l ,[640 , 480]).astype(int)) ,
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.5 , 
                        (255,255,255),
                        1,
                        cv2.LINE_AA
                       )            

            

            


            cv2.putText(frame , str(angle_right) , 
                       tuple(np.multiply(knee_r ,[640 , 480]).astype(int)) ,
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.5 , 
                        (255,255,255),
                        1,
                        cv2.LINE_AA
                       )
            
            cv2.putText(frame , f'{final_type}, {final_sev}' , 
                        (7,50) ,
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.5 , 
                        (255,255,255),
                        2,
                        cv2.LINE_AA
                       )





        cv2.imshow('Pose with Mediapipe' , frame)




⚠️ Image not found or path incorrect


AttributeError: 'NoneType' object has no attribute 'shape'